In [1]:
Importando dados Covid 19

SyntaxError: invalid syntax (1165714557.py, line 1)

In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
covid_complete = pd.read_csv('/kaggle/input/corona-virus-report/covid_19_clean_complete.csv')
covid_complete.head(20)

a. Quantas linhas e quantas colunas possui esse banco?

In [ ]:
covid_complete.info()

Possui 49067 linhas e 10 colunas

b. Quais são as colunas

In [ ]:
covid_complete.columns

As colunas são: Province/State, Country/Region, Lat, Long, Date, Confirmed,
       Deaths, Recovered, Active, WHO Region

c. O tipo de dado de cada coluna

In [ ]:
covid_complete.info()

Province/State    object 
  Country/Region  object 
  Lat             float64
  Long            float64
  Date            object 
  Confirmed       int64  
  Deaths          int64  
  Recovered       int64  
  Active          int64  
  WHO Region      object 


d. A coluna de datas deve ser transformada de “Object” para “datetime64”,
utilizando o comando “pd.todatetime( )”

In [ ]:
covid_complete.Date = pd.to_datetime(covid_complete.Date)


In [ ]:
covid_complete.info()

e. Informações estatísticas sobre o banco de dados

In [ ]:

mean_df = covid_complete.mean()
print(mean_df)

f. Se necessário, transformar uma coluna pertinente para formato de data

g. Quais colunas apresentam NaN

In [ ]:
for column in covid_complete:
    num_nan = covid_complete[column].isna().sum()
    print('{} --> {}'.format(column,num_nan))

Há "NaN" apenas na primeira coluna (Província/Estado) com 34404 NaN

2. Crie um dataframe que contenha apenas as 5 províncias da China que mais registraram casos.

In [ ]:
china_regions_totals = covid_complete[covid_complete['Country/Region'] == 'China']


a. Descubra qual o nome das províncias chinesas contidas no dataset
b. Retire do banco de dados apenas as informações dessas províncias
c. Tome apenas as informações das features: “Confirmed”, “Active”, “Deaths”,
“Recovered”
d. Agrupe o novo dataframe a partir dessas províncias. Qual a função de
agrupamento que deve ser usada?


In [ ]:
china_regions_totals = china_regions_totals.loc[:,['Province/State','Confirmed','Deaths','Recovered','Active']]
china_regions_totals = china_regions_totals.groupby('Province/State').sum().sort_values('Confirmed', ascending = False)
china_top5 = china_regions_totals.head(5)
china_top5

As cidades da China com mais casos são: Hubei, Guangdong, Henan, Zhejiang, Hunan

3. A coluna “Province/State” contém muitos valores faltando. Para descartá-la perdendo o mínimo de informação, inclua na coluna “Country/Region” o nome das regiões junto ao nome dos países em que existem mais de uma província registrada no banco de dados, utilizando o método “.apply”. Como por exemplo, juntar o nome China com a região pertinente: “China_Hubei”, “China_Guangdong”.


a. Elabore uma função que tenha uma linha do banco de dados como argumento e, se a coluna “Province/State” não for valor faltante, concatenar seu valor com coluna “Country/Region”
i. Utilize o método “pandas.notna” para averiguar de “Province/State” é valor faltante

In [ ]:
def noprovince(row):
    if pd.notna(row['Province/State']):
        row['Country/Region'] = row['Country/Region'] + '_' + row['Province/State']
    return row

b. Faça uma cópia do banco de dados para poder retirar informações sem perder o banco de dados original

In [ ]:
covid_noprovince = covid_complete.copy()

c. Aplique, por meio do método “apply”, a função criada no item a

In [ ]:
covid_noprovince = covid_noprovince.apply(noprovince, axis = 1)
covid_noprovince = covid_noprovince.drop(['Province/State'], axis = 1)
covid_noprovince.head(20)

d. Exclua a coluna “Province/State” do novo dataframe


In [ ]:
covid_noprovince.loc[covid_noprovince['Country/Region'] == 'China']

4. Importe o arquivo "world meter data.csv" e, a partir de seus dados de população e continentes, faça um ranking de maior número de mortes por milhão de habitantes entre os continentes.

a. Importar o banco de dados "world meter data.csv"

In [ ]:
worldometer = pd.read_csv('/kaggle/input/corona-virus-report/worldometer_data.csv')


b. Tome apenas as informações de população, país e continentes deste banco de
dados
c.Agrupe o “covid_19_clean_complete” por países. Deve-se usar o dataframe do
item “c)” ou o original?
d. Mescle os dois dataframes, a partir dos países, contendo apenas as informações de mortes, população e continente


In [ ]:
covid_comp_deaths = covid_complete[['Country/Region','Deaths']].groupby('Country/Region').sum()
covid_world = covid_comp_deaths.merge(worldometer[['Country/Region','Population','Continent']],
                                            on = 'Country/Region')
covid_world

e. Agrupe o novo dataset agora pelo continente

In [ ]:
covid_world_deaths = covid_world.groupby('Continent', as_index = False).sum()

f. Crie uma nova coluna contendo o número de mortes por milhão dentro dos
continentes (Utilize a fórmula: (mortes/população)*10^6)

In [ ]:
covid_world_deaths['Deaths_by_Million'] = (covid_world_deaths['Deaths']/covid_world_deaths['Population'])*10**6

g. Faça um ranking do número de mortes por milhão de habitantes entre os
continentes

In [ ]:
covid_world_deaths = covid_world_deaths.sort_values(by = 'Deaths_by_Million', ascending = False)
covid_world_deaths['Ranking'] =[x for x in range(1,len(covid_world_deaths)+1)]
covid_world_deaths = covid_world_deaths.set_index("Ranking")
covid_world_deaths
                       

# Visualização de dados

##  Line Chart

1. Criar gráfico linechart que indique a progressão no tempo das mortes nas regiões: Américas, Eastern Mediterran e Europe (OBS: Utilize apenas o arquivo “covid_19_clean_complete.csv”)
a. Importar bibliotecas matplotlib e seaborn, para retirar a necessidade de criar arquivos para os gráficos, digitar comando “%matplotlib inline”

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

b. Realizar agrupamento utilizando as colunas “Date” e “WHO Region”, utilizando “sum” como função de agregação

In [ ]:
covid_who = covid_complete.groupby(by = ['Date','WHO Region']).sum().reset_index(level = 'Date')
covid_who.head(15)

c. Plotar gráficos que contenham as progressões das regiões exigidas
d. adicionar nomes para eixos, título do gráfico, grid e legendas para as linhas

In [ ]:
covid_who.Date = pd.to_datetime(covid_who.Date)
covid_who.info()

In [ ]:
sns.set_style("darkgrid")


plt.plot('Date','Deaths', data = covid_who.loc[covid_who.index == "Eastern Mediterranean"], c = 'y');
plt.plot('Date','Deaths', data = covid_who.loc[covid_who.index == "Europe"], c = 'b');
plt.plot('Date','Deaths', data = covid_who.loc[covid_who.index == "Americas"], c = 'g');
plt.xlabel('Date')
plt.ylabel('Deaths')
plt.legend(["Eastern Mediterranean","Europe","Americas"]);

e. Analisar gráfico de acordo com as perguntas feitas

1. Quais das regiões tiveram mais mortes?

As regiões que apresentaram mais mortes foi na America seguida da Europa

2. Qual o momento em que cada progressão começa a subir?

Europa - Março de 2020
America - Abril de 2020
Mediterrâneo- Não foi acentuado, começou a ter um leve aumento a partir de Abril de 2020

## Bar Charts

1. Agora deve-se produzir um gráfico bar chart, indicando o número de mortes em cada continente (OBS: utilize o dataframe criado no exercício 4.1 - 4)

a. Editar o tamanho do gráfico, como sugestão, coloque o tamanho (10,6)
b. Plotar gráfico bar chart das mortes nos continentes
c. editar título e nome dos eixos


In [ ]:
plt.figure(figsize =(10,6))
plt.bar('Continent','Deaths', data = covid_world );

plt.title('Continet Deaths')
plt.ylabel('Deaths')

d. Responder perguntas com base no gráfico
1. Quais os 2 continentes com maior número de mortes?

Ámerica do sul e Europa

2. Qual o continente com menor número de mortes?

Oceania

3. Qual sua hipótese para responder às perguntas anteriores?


Oceania, por apresentar um sistema de saúde melhor, conseguiu até aquele momento "reter"melhor os casos. E como a Europa contém muitos idosos, fez com que os casos aumentasse muito logo no ínício, ocupando muito os leitos e levando a um maior número de mortos.

## Scatter Plots

1. Descobrir se há relação entre o número de mortes por milhão e a quantidade de habitantes, nos 3 continentes com maior média de mortes por milhão de habitantes, por meio de scatter plot (OBS: Utilizar somente arquivo “worldometer_data.csv”)
a. Agrupar o dataframe pela coluna “Continent”, retirar apenas os dados de "Deaths/1 M pop", utilizar “mean” como função de agregação e ordenar para descobrir os 3 continentes com maior valor
b. Obter apenas os dados do dataframe, correspondentes a esses continentes
c. Criar Scatterplot sendo o eixo x mostrando as mortes por milhão, o eixo y, a
população total e representar os 3 continentes com cores diferentes



In [ ]:
world_most_deaths = worldometer[['Continent','Deaths/1M pop']].groupby('Continent', as_index = False).mean().sort_values('Deaths/1M pop',ascending = False)
world_most_deaths


In [ ]:
pop_x_deaths = worldometer.loc[(worldometer['Continent'] == 'South America')|
                               (worldometer['Continent'] =='Europe')|
                               (worldometer['Continent'] =='North America'),
                              ['Continent','Population','Deaths/1M pop']]
pop_x_deaths

In [ ]:
plt.figure(figsize =(12,6))
plt.title('Relação entre Mortes por milhão e população total do país')

sns.scatterplot(x = 'Population',
               y = 'Deaths/1M pop',
               hue = 'Continent',
               data = pop_x_deaths);

1. Onde se concentram a maioria dos dados?

Ámerica do Sul
Europa 

2. No segundo continentes por mais mortes por milhão, é possível perceber
alguma relação entre as features

3. Pelo gráfico, é possível enxergar relação entre essas colunas do banco de
dados?

Que em lugares menores, há maior concentração de pessoas

## Heatmap de Correlação

1. Será utilizado o método DataFrame.corr() do pandas para gerar os valores de correlação, então o aluno deverá produzir um heatmap para a visualização dos valores gerados

a. Criar uma variável para armazenar os valores de correlação
b. Criar um heatmap, com título, utilizando a variável criada no item anterior
c. Escolher duas features para analisar seus valores de correlação (Sugestão:
“Population”, “Teste/1M pop”)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
plt.title('Correlação entre colunas')
sns.heatmap(worldometer.corr(), annot=True, square=True, cmap='coolwarm');